In [1]:
import pandas as pd
import os

In [2]:
team_splits = pd.read_pickle(os.path.join('data', 'team_splits_periods.pickle'))
scoring = pd.read_pickle(os.path.join('data', 'scoring.pickle'))

In [3]:
ducks = team_splits.loc[team_splits["name"] == "Anaheim Ducks", :]
ducks = ducks.sort_index()
ducks.head(10)

,name,L,OL,T,W
month,,,,,
2006-01,Anaheim Ducks,5.0,2.0,NaN,4.0
2006-02,Anaheim Ducks,5.0,2.0,NaN,5.0
2006-03,Anaheim Ducks,3.0,2.0,NaN,10.0
2006-04,Anaheim Ducks,0.0,2.0,NaN,1.0
2006-10,Anaheim Ducks,0.0,3.0,NaN,9.0
2006-11,Anaheim Ducks,2.0,3.0,NaN,10.0
2006-12,Anaheim Ducks,5.0,0.0,NaN,9.0
2007-01,Anaheim Ducks,4.0,1.0,NaN,8.0
2007-02,Anaheim Ducks,2.0,1.0,NaN,10.0


In [4]:
ducks.resample("A").sum()

,L,OL,T,W
month,,,,
2006,20.0,14.0,0.0,48.0
2007,27.0,8.0,0.0,47.0
2008,33.0,7.0,0.0,42.0
2009,32.0,11.0,0.0,39.0
2010,30.0,5.0,0.0,47.0
2011,36.0,12.0,0.0,34.0


In [5]:
ducks.index = ducks.to_timestamp().index.map(
    lambda x: 
    x + pd.DateOffset(years=1) if x.month < 5
    else x
)
ducks = ducks.sort_index()
ducks.head()

,name,L,OL,T,W
month,,,,,
2006-10-01,Anaheim Ducks,0.0,3.0,NaN,9.0
2006-11-01,Anaheim Ducks,2.0,3.0,NaN,10.0
2006-12-01,Anaheim Ducks,5.0,0.0,NaN,9.0
2007-01-01,Anaheim Ducks,5.0,2.0,NaN,4.0
2007-02-01,Anaheim Ducks,5.0,2.0,NaN,5.0


In [6]:
resampler = ducks.resample("A-JUN")
resampler

In [7]:
resampler.sum()

,L,OL,T,W
month,,,,
2007-06-30,20.0,14.0,0.0,48.0
2008-06-30,27.0,8.0,0.0,47.0
2009-06-30,33.0,7.0,0.0,42.0
2010-06-30,32.0,11.0,0.0,39.0
2011-06-30,30.0,5.0,0.0,47.0
2012-06-30,36.0,12.0,0.0,34.0


In [8]:
# We can't to do this: resampler.expanding().sum()
res = resampler.apply(
        lambda x: x.reset_index()[["W", "L"]]
                   .expanding().sum()
)
res.head(8)

W     L
month                   
2007-06-30 0   9.0   0.0
           1  19.0   2.0
           2  28.0   7.0
           3  32.0  12.0
           4  37.0  17.0
           5  47.0  20.0
           6  48.0  20.0
2008-06-30 0   1.0   1.0

In [9]:
res.index = ducks.index
res.head()

,W,L
month,,
2006-10-01,9.0,0.0
2006-11-01,19.0,2.0
2006-12-01,28.0,7.0
2007-01-01,32.0,12.0
2007-02-01,37.0,17.0


In [10]:
final = res.asfreq(pd.tseries.offsets.MonthBegin())
final.head(10)

,W,L
month,,
2006-10-01,9.0,0.0
2006-11-01,19.0,2.0
2006-12-01,28.0,7.0
2007-01-01,32.0,12.0
2007-02-01,37.0,17.0
2007-03-01,47.0,20.0
2007-04-01,48.0,20.0
2007-05-01,NaN,NaN
2007-06-01,NaN,NaN


In [11]:
fdf = final.reset_index()
fdf.head()

,month,W,L
0,2006-10-01,9.0,0.0
1,2006-11-01,19.0,2.0
2,2006-12-01,28.0,7.0
3,2007-01-01,32.0,12.0
4,2007-02-01,37.0,17.0


In [12]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [13]:
from bokeh.models.sources import ColumnDataSource as cds

In [14]:
source = cds(fdf)
p = figure(x_axis_type="datetime", height=500, width=500)
p.line(source=source, x='month', y="W", color="green")
p.line(source=source, x='month', y="L", color="red");

In [15]:
show(p)

In [16]:
source = cds(fdf)
p = figure(x_axis_type="datetime", height=500, width=500)
p.line(source=source, x='month', y="W", color="green", legend="Wins")
p.line(source=source, x='month', y="L", color="red", legend="Loses")
p.legend.click_policy="hide";

In [17]:
show(p)

In [18]:
source = cds(fdf)
p = figure(x_axis_type="datetime", height=500, width=500)
p.line(source=source, x='month', y="W", color="green", legend="Wins")
p.line(source=source, x='month', y="L", color="red", legend="Loses")
p.legend.click_policy="hide"
p.title.text = "Anaheim Ducks Performance by Season"
p.title.text_font_size = "25px"
p.title.align = "center";

In [19]:
show(p)

In [20]:
wayne = scoring.set_index("playerID").loc['gretzwa01']
wayne.head()

,year,tmID,GP,G,A,Pts,SOG
playerID,,,,,,,
gretzwa01,1980,EDM,80.0,55.0,109.0,164.0,261.0
gretzwa01,1981,EDM,80.0,92.0,120.0,212.0,369.0
gretzwa01,1982,EDM,80.0,71.0,125.0,196.0,348.0
gretzwa01,1983,EDM,74.0,87.0,118.0,205.0,324.0
gretzwa01,1984,EDM,80.0,73.0,135.0,208.0,358.0


In [21]:
wayne = wayne[['year', 'tmID', 'GP', 'Pts']]
wayne = wayne.assign(pts_per_game = lambda x: x['Pts'] / x['GP'])
wayne.head()

,year,tmID,GP,Pts,pts_per_game
playerID,,,,,
gretzwa01,1980,EDM,80.0,164.0,2.05000
gretzwa01,1981,EDM,80.0,212.0,2.65000
gretzwa01,1982,EDM,80.0,196.0,2.45000
gretzwa01,1983,EDM,74.0,205.0,2.77027
gretzwa01,1984,EDM,80.0,208.0,2.60000


In [22]:
wayne.tmID.head()

playerID
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
Name: tmID, dtype: category
Categories (37, object): ['ANA', 'AND', 'ATL', 'BOS', ..., 'VAN', 'WAS', 'WIN', 'WPG']

In [23]:
wayne.loc[:, "tmID"] = wayne.tmID.cat.remove_unused_categories()
wayne.tmID.head()

playerID
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
gretzwa01    EDM
Name: tmID, dtype: category
Categories (4, object): ['EDM', 'LAK', 'NYR', 'STL']

In [24]:
wayne.loc[:, "tmCode"] = wayne["tmID"].cat.codes
wayne.sample(5)

,year,tmID,GP,Pts,pts_per_game,tmCode
playerID,,,,,,
gretzwa01,1991,LAK,74.0,121.0,1.635135,1
gretzwa01,1994,LAK,48.0,48.0,1.000000,1
gretzwa01,1981,EDM,80.0,212.0,2.650000,0
gretzwa01,1988,LAK,78.0,168.0,2.153846,1
gretzwa01,1983,EDM,74.0,205.0,2.770270,0


In [25]:
wayne.loc[:, "year"] = pd.to_datetime(wayne.year, format="%Y")

In [26]:
gp_max = wayne.GP.max()
gp_min = wayne.GP.min()
pts_per_game_max = wayne.pts_per_game.max()
pts_per_game_min = wayne.pts_per_game.min()

In [27]:
wayne.loc[:, "height"] = wayne.GP / gp_max
wayne.loc[:, "bottom"] = wayne.tmCode
wayne.loc[:, "top"] = wayne.bottom + wayne.height
wayne.head()

,year,tmID,GP,Pts,pts_per_game,tmCode,height,bottom,top
playerID,,,,,,,,,
gretzwa01,1980-01-01,EDM,80.0,164.0,2.05000,0,0.975610,0,0.975610
gretzwa01,1981-01-01,EDM,80.0,212.0,2.65000,0,0.975610,0,0.975610
gretzwa01,1982-01-01,EDM,80.0,196.0,2.45000,0,0.975610,0,0.975610
gretzwa01,1983-01-01,EDM,74.0,205.0,2.77027,0,0.902439,0,0.902439
gretzwa01,1984-01-01,EDM,80.0,208.0,2.60000,0,0.975610,0,0.975610


In [28]:
src = cds(wayne)
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=1, source=src);

In [29]:
show(p)

In [30]:
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=1, source=src)
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

In [31]:
show(p)

In [32]:
reordered = wayne.sort_values('year').tmID.unique()
reordered

['EDM', 'LAK', 'STL', 'NYR']
Categories (4, object): ['EDM', 'LAK', 'NYR', 'STL']

In [33]:
wayne.loc[:, "tmID"] = wayne.tmID.cat.reorder_categories(reordered)
wayne.loc[:, "tmCode"] = wayne["tmID"].cat.codes
wayne.loc[:, "bottom"] = wayne.tmCode
wayne.loc[:, "top"] = wayne.bottom + wayne.height

In [34]:
width = 320 * 24 * 60 * 60 * 1000

In [35]:
src = cds(wayne)
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=width, source=src)
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

In [36]:
show(p)

In [37]:
wayne.loc[:, "bottom"] = wayne.tmCode - (wayne.height / 2.0 ) + 0.5
wayne.loc[:, "top"] = wayne.bottom + wayne.height

In [38]:
src = cds(wayne)
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=width, source=src)
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

In [39]:
show(p)

In [40]:
wayne.loc[:, "height"] = wayne.GP / gp_max * 0.8
wayne.loc[:, "bottom"] = wayne.tmCode - (wayne.height / 2.0 ) + 0.5
wayne.loc[:, "top"] = wayne.bottom + wayne.height

In [41]:
src = cds(wayne)
p = figure(x_axis_type='datetime',
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom',
       top='top', x='year',
       width=width, source=src)
p.xaxis.major_label_text_font_size = "12pt"
p.yaxis.major_label_text_font_size = "12pt"

In [42]:
show(p)

In [43]:
from bokeh.models import LinearColorMapper
color_mapper = LinearColorMapper(
    palette="Plasma256", 
    low=pts_per_game_min, 
    high=pts_per_game_max)

In [44]:
src = cds(wayne)
p = figure(x_axis_type='datetime', 
           height=500, width=500,
           y_range=list(wayne.tmID.cat.categories))
p.vbar(bottom='bottom', top='top',
       x='year', width=width,
       color={'field': 'pts_per_game', 'transform': color_mapper},
       source=src);

In [45]:
show(p)

In [46]:
from bokeh.models import ColorBar
src = cds(wayne)
p = figure(x_axis_type='datetime', 
           width=500,
           height=500, 
           y_range=list(wayne.tmID.cat.categories)
)
p.vbar(bottom='bottom', top='top', x='year', width=width,
       color={'field': 'pts_per_game', 'transform': color_mapper}, source=src)
color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8, border_line_color=None, 
                     location=(0,0)
)

p.add_layout(color_bar, 'right');

In [47]:
show(p)

In [48]:
from bokeh.models.tools import HoverTool
hover = HoverTool(
    tooltips=[
        ("Points per game", "@pts_per_game"),
        ("Games played", "@GP")
    ]
)

In [49]:
src = cds(wayne)
p = figure(x_axis_type='datetime', 
           width=500,
           height=500, 
           y_range=list(wayne.tmID.cat.categories)
)
p.vbar(bottom='bottom', top='top', x='year', width=width,
       color={'field': 'pts_per_game', 'transform': color_mapper}, source=src)
color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8, border_line_color=None, 
                     location=(0,0)
)

p.add_layout(color_bar, 'right')
p.add_tools(hover);

In [50]:
show(p)